In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv('D:\datasets\spotify_songs.csv')
data.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


In [2]:
data = data.drop(columns=['track_popularity','track_album_release_date','playlist_name','playlist_id', 'track_album_id', 'track_album_name','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','danceability','energy','key'])

In [3]:
data.head(1)

,track_id,track_name,track_artist,lyrics,playlist_genre,playlist_subgenre,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,rock,classic rock,tl


In [4]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [5]:
data['artist'] = data['track_artist'].apply(collapse)
data['genre'] = data['playlist_genre'].apply(collapse)

In [6]:
data['tags'] = data['artist'] + data['genre']
data['tags'][0]

['B',
 'a',
 'r',
 'b',
 'i',
 'e',
 "'",
 's',
 '',
 'C',
 'r',
 'a',
 'd',
 'l',
 'e',
 'r',
 'o',
 'c',
 'k']

In [7]:
data['tags'] = data['tags'].apply(lambda x: " ".join(x))

In [8]:
data.to_csv('songs_data.csv')

In [9]:
#data = data.head(10000)
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
cv = CountVectorizer(min_df=0, max_features=500,stop_words='english', token_pattern='(?u)\\b\\w+\\b')
#tf = TfidfVectorizer(analyzer='word',min_df=0, stop_words='english', token_pattern='(?u)\\b\\w+\\b', max_features=5000)

In [10]:
data['tags']

0                     B a r b i e ' s  C r a d l e r o c k
1                         S t e a d y  R o l l i n r o c k
2                          B e l l  B i v  D e V o e r & b
3                               C e e L o  G r e e n r & b
4                                            K A R D p o p
                               ...                        
18449                                  Q u l i n e z e d m
18450                           N i c k i  M i n a j p o p
18451    P o n d e r o s a  T w i n s  P l u s  O n e r...
18452                               F a t h e r  M C r & b
18453                          M o o n s t a r 8 8 r o c k
Name: tags, Length: 18454, dtype: object

In [11]:
#vector = cv.fit_transform(data['tags']).toarray()
vector = cv.fit_transform(data['tags']).toarray()

In [12]:
vector.shape

(18454, 37)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity.size

340550116

In [14]:
similarities = {}
for i in range(len(similarity)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs.
    similar_indices = similarity[i].argsort()[:-50:-1]
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[data['track_name'].iloc[i]] = [(similarity[i][x], data['track_name'][x], data['track_artist'][x]) for x in similar_indices][1:]

In [15]:
import pickle
file = "similarity.pkl"
fileobj = open(file, 'wb')
pickle.dump(similarities, fileobj)
fileobj.close()

In [16]:
class SongRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def get_info(self, recom_song):
        rec_items = len(recom_song)
        songs = []
        artist_list = []
        for i in range(rec_items):
            songs.append(recom_song[i][1])
            artist_list.append(recom_song[i][2])
            df = data[data['track_name'] == recom_song[i][1]]
            #lyrics.append(df['Lyrics'])
            #print(f"{recom_song[i][1]} by {recom_song[i][2]}")
        return songs, artist_list
        
    def recommend(self, song):
        recommendations = []
        # Get artist to find recommendations for
        artist_df = data[data['track_name'] == song]
        artist = artist_df['track_artist']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][1:6]
        songs, artists = self.get_info(recom_song=recom_song)
        for x in range(len(artists)):
            d = {}
            d['title'] = songs[x]
            d['artist'] = artists[x]
            #d['lyrics'] = lyrics[x].iloc[0]
            recommendations.append(d)
        return recommendations

In [17]:
def getRecommendations(song):
    return SongRecommender(similarities).recommend(song)

In [18]:
getRecommendations("Ignite")

[{'title': 'All Falls Down (feat. Juliander)', 'artist': 'Alan Walker'},
 {'title': 'Faded - Dash Berlin Remix', 'artist': 'Alan Walker'},
 {'title': 'Diamond Heart', 'artist': 'Alan Walker'},
 {'title': 'Sing Me to Sleep', 'artist': 'Alan Walker'},
 {'title': 'Darkside', 'artist': 'Alan Walker'}]